In [1]:
import json
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format
import torch

In [10]:
from huggingface_hub import login
login()

In [8]:
transcripts = {}
# with open("./interview_transcripts_by_turkers.csv") as file:
#         for curr in file:
#                 currsplit = curr.split(",")
#                 concat_string = ",".join(currsplit[1:])
#                 concat_string = concat_string.split("|")
#                 transcripts[(currsplit[0])] = concat_string
# with open("processed_transcript.json", 'w') as fp:
#     transcripts_data= []
#     for curr_transcript_key in transcripts.keys():
#         curr_transcript = []
#         for curr_line in transcripts[curr_transcript_key]:
#             curr_line = curr_line.split(": ")
#             curr_transcript.append({'role': "assistant" if curr_line[0] == "Interviewer" else "user", 'content': curr_line[-1]})
#         transcripts_data.append({"messages":curr_transcript})
#     json.dump(transcripts_data, fp)

In [5]:
with open("processed_transcript.json", 'r') as jsonfile:
    data = json.load(jsonfile)

In [2]:
ds = load_dataset('json', data_files='processed_transcript.json',split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:

 
args = TrainingArguments(
    output_dir="llama-3.2-1b-full-ft-interview", # directory to save and repository id
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=8,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler                 
    report_to="tensorboard",                # report metrics to tensorboard
)

In [4]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# model, tokenizer = setup_chat_format(model, tokenizer)

In [5]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=ds,
#     peft_config=peft_config,
#     max_seq_length=4096,
#     tokenizer=tokenizer,
#     packing=True,
#     dataset_kwargs={
#         "add_special_tokens": False,  # We template with special tokens
#         "append_concat_token": False, # No need to add additional separator token
#     }
)

/home/hice1/bsong74/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [7]:
trainer.train()

Step,Training Loss
10,5.342200
20,2.814000


TrainOutput(global_step=27, training_loss=3.5542578520598234, metrics={'train_runtime': 79.8644, 'train_samples_per_second': 5.184, 'train_steps_per_second': 0.338, 'total_flos': 2523142098518016.0, 'train_loss': 3.5542578520598234, 'epoch': 3.0})